In [2]:
%%capture
!pip install transformers datasets

In [3]:
import os
import random
import torch
import numpy as np

def seed_all(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_all(42)

In [4]:
from datasets import load_dataset, load_metric

task = "cola"
dataset = load_dataset("glue", task)
metric = load_metric("glue", task)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/377k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [5]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [6]:
dataset["train"][0]

{'idx': 0,
 'label': 1,
 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}

In [7]:
dataset.pop("test")

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 1063
})

In [8]:
from transformers import TrainingArguments

batch_size = 64

args = TrainingArguments(
    "test-glue",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="matthews_correlation",
)

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer

device = "cuda" if torch.cuda.is_available() else "cpu"

def experiment(model_name):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=2
    ).to(torch.device(device))
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    encoded_dataset = dataset.map(lambda example: tokenizer(example['sentence'], truncation=True), batched=True)
    encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"], device=device)
    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer, 
        compute_metrics=compute_metrics,
    )
    trainer.train()
    result = trainer.evaluate()["eval_matthews_correlation"]
    return (trainer.model, result)

In [11]:
results = {}
model_names = ["bert-base-uncased", "distilbert-base-uncased", "distilroberta-base", 'roberta-base', 'albert-base-v2']

for model_name in model_names:
    print(f"=====Tuning {model_name}=====")
    results[model_name] = experiment(model_name)

=====Tuning bert-base-uncased=====


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

The following columns in the training set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running training *****
  Num examples = 8551
  Num Epochs = 4
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 536


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.618123,0.000000
2,No log,0.633487,0.000000
3,No log,0.618976,0.000000
4,0.615100,0.618088,0.000000


The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
Saving model checkpoint to test-glue/checkpoint-134
Configuration saved in test-glue/checkpoint-134/config.json
Model weights saved in test-glue/checkpoint-134/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-134/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-134/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 64
/usr/loca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [12]:
for model_name, (_, score) in results.items():
    print(f"{model_name}: {score:.5f}")

bert-base-uncased: 0.59903
distilbert-base-uncased: 0.52506
distilroberta-base: 0.58764
roberta-base: 0.59617
albert-base-v2: 0.00000


In [13]:
best_model_name = max(results, key=lambda k: results[k][1])
best_model = results[best_model_name][0]
file_name = f"custom-{best_model_name}"
best_model.save_pretrained(file_name)

Configuration saved in custom-bert-base-uncased/config.json
Model weights saved in custom-bert-base-uncased/pytorch_model.bin


In [14]:
!zip -r custom-bert-base-uncased.zip custom-bert-base-uncased/

  adding: custom-bert-base-uncased/ (stored 0%)
  adding: custom-bert-base-uncased/pytorch_model.bin (deflated 7%)
  adding: custom-bert-base-uncased/config.json (deflated 49%)


In [15]:
from google.colab import files

files.download(f"{file_name}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>